**Checkpoint II**

In [79]:
import pandas as pd
from datetime import date, timedelta
from sklearn.compose import make_column_transformer
from sklearn.linear_model import LinearRegression
from sklearn.pipeline import make_pipeline
from sklearn.preprocessing import OneHotEncoder, StandardScaler, OrdinalEncoder

In [80]:
train = pd.read_csv("train.csv", parse_dates=['date'])
test = pd.read_csv("test.csv", parse_dates=['date'])
oil = pd.read_csv("oil.csv", parse_dates=['date'])
holidays_events = pd.read_csv("holidays_events.csv", parse_dates=['date'])
transactions = pd.read_csv("transactions.csv", parse_dates=['date'])
stores = pd.read_csv("stores.csv")

In [81]:
print(train.head())
x,y=train.shape
print("\nNo of rows in training data initially: ",x)
print("No of cols in training data initially: ",y)

   id       date  store_nbr      family  sales  onpromotion
0   0 2013-01-01          1  AUTOMOTIVE    0.0            0
1   1 2013-01-01          1   BABY CARE    0.0            0
2   2 2013-01-01          1      BEAUTY    0.0            0
3   3 2013-01-01          1   BEVERAGES    0.0            0
4   4 2013-01-01          1       BOOKS    0.0            0

No of rows in training data initially:  3000888
No of cols in training data initially:  6


In [82]:
train.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 3000888 entries, 0 to 3000887
Data columns (total 6 columns):
 #   Column       Dtype         
---  ------       -----         
 0   id           int64         
 1   date         datetime64[ns]
 2   store_nbr    int64         
 3   family       object        
 4   sales        float64       
 5   onpromotion  int64         
dtypes: datetime64[ns](1), float64(1), int64(3), object(1)
memory usage: 137.4+ MB


In [83]:
calendar = pd.DataFrame(index=pd.date_range(train.date.min(), test.date.max()))
# days of week
calendar['weekday'] = calendar.index.dayofweek 
calendar

,weekday
2013-01-01,1
2013-01-02,2
2013-01-03,3
2013-01-04,4
2013-01-05,5
...,...
2017-08-27,6
2017-08-28,0
2017-08-29,1
2017-08-30,2


In [84]:
train_ext = train.merge(stores, on='store_nbr', how='left')
train_ext = train_ext.merge(transactions, on=['date', 'store_nbr'], how='left')
# train_ext = train_ext.rename(columns={"type": "store_type"})
# train_ext['date'] = train_ext['date'].astype('datetime64[ns]')
train_ext['date'] = train_ext.date.dt.to_period('D')
train_ext = train_ext.set_index(['store_nbr', 'family', 'date']).sort_index()
train_ext

id  sales  onpromotion       city  \
store_nbr family     date                                                 
1         AUTOMOTIVE 2013-01-01        0    0.0            0      Quito   
                     2013-01-02     1782    2.0            0      Quito   
                     2013-01-03     3564    3.0            0      Quito   
                     2013-01-04     5346    3.0            0      Quito   
                     2013-01-05     7128    5.0            0      Quito   
...                                  ...    ...          ...        ...   
54        SEAFOOD    2017-08-11  2993627    0.0            0  El Carmen   
                     2017-08-12  2995409    1.0            1  El Carmen   
                     2017-08-13  2997191    2.0            0  El Carmen   
                     2017-08-14  2998973    0.0            0  El Carmen   
                     2017-08-15  3000755    3.0            0  El Carmen   

                                     state type  cluster  transactions  
store_nbr family     date                                               
1         AUTOMOTIVE 2013-01-01  Pichincha    D       13           NaN  
                     2013-01-02  Pichincha    D       13        2111.0  
                     2013-01-03  Pichincha    D       13        1833.0  
                     2013-01-04  Pichincha    D       13        1863.0  
                     2013-01-05  Pichincha    D       13        1509.0  
...                                    ...  ...      ...           ...  
54        SEAFOOD    2017-08-11     Manabi    C        3         768.0  
                     2017-08-12     Manabi    C        3         903.0  
                     2017-08-13     Manabi    C        3        1054.0  
                     2017-08-14     Manabi    C        3         818.0  
                     2017-08-15     Manabi    C        3         802.0  

[3000888 rows x 8 columns]

In [85]:
holidays_events = holidays_events[holidays_events.locale=='National']
holidays_events = holidays_events.set_index('date').sort_index()
# keep only one event per day
holidays_events = holidays_events.groupby(holidays_events.index).first()
holidays_events.drop(columns=['locale', 'locale_name', 'description'], inplace=True)
holidays_events

,type
date,
2012-08-10,Holiday
2012-10-09,Holiday
2012-10-12,Transfer
2012-11-02,Holiday
2012-11-03,Holiday
...,...
2017-12-22,Additional
2017-12-23,Additional
2017-12-24,Additional


In [86]:
def compute_workdays(df, dofw_col):
    df['workday'] = True
    # exclude week-ends
    df.loc[df[dofw_col] > 4, 'workday'] = False
    # friday bridges are not working days
    df.loc[df.type=='Bridge', 'workday'] = False
    # some bridges are recovered by working at weekends
    df.loc[df.type=='Work Day', 'workday'] = True
    # handling Transfered events
    df.loc[df.type=='Transfer', 'workday'] = False
    df.loc[(df.type=='Holiday')&(df.transferred==False), 'workday'] = False
    df.loc[(df.type=='Holiday')&(df.transferred==True ), 'workday'] = True
    return df

In [87]:
calendar = calendar.merge(holidays_events, how='left', left_index=True, right_index=True)
calendar = compute_workdays(calendar, 'weekday')
calendar['workday'] = calendar['workday'] * 1
# calendar.index = calendar.index.to_period('D')

AttributeError: 'DataFrame' object has no attribute 'transferred'

In [ ]:
calendar.head(60)

,weekday,type,locale,locale_name,description,transferred,workday
2013-01-01,1,Holiday,National,Ecuador,Primer dia del ano,False,0
2013-01-02,2,NaN,NaN,NaN,NaN,NaN,1
2013-01-03,3,NaN,NaN,NaN,NaN,NaN,1
2013-01-04,4,NaN,NaN,NaN,NaN,NaN,1
2013-01-05,5,Work Day,National,Ecuador,Recupero puente Navidad,False,1
2013-01-06,6,NaN,NaN,NaN,NaN,NaN,0
2013-01-07,0,NaN,NaN,NaN,NaN,NaN,1
2013-01-08,1,NaN,NaN,NaN,NaN,NaN,1
2013-01-09,2,NaN,NaN,NaN,NaN,NaN,1
2013-01-10,3,NaN,NaN,NaN,NaN,NaN,1


# ___________________BREAK HERE _________________

In [ ]:
training_data_df = pd.read_csv("train.csv")
training_data_df['date'] = pd.to_datetime(training_data_df['date'], format='%Y-%m-%d')

#remove entries bewtween the day before the earthquake struck and 3 months after.
training_data_df = training_data_df.loc[(training_data_df['date'] < '2016-04-15')
                     | (training_data_df['date'] > '2016-05-15')]
                     
id_col = training_data_df[["id"]]
target = "sales"
y_data = training_data_df[[target]]
X_data = training_data_df.drop(target, axis=1)
X_data

,id,date,store_nbr,family,onpromotion
0,0,2013-01-01,1,AUTOMOTIVE,0
1,1,2013-01-01,1,BABY CARE,0
2,2,2013-01-01,1,BEAUTY,0
3,3,2013-01-01,1,BEVERAGES,0
4,4,2013-01-01,1,BOOKS,0
...,...,...,...,...,...
3000883,3000883,2017-08-15,9,POULTRY,0
3000884,3000884,2017-08-15,9,PREPARED FOODS,1
3000885,3000885,2017-08-15,9,PRODUCE,148
3000886,3000886,2017-08-15,9,SCHOOL AND OFFICE SUPPLIES,8


In [ ]:
x_test = pd.read_csv("test.csv")
id_col = x_test[["id"]]
x_test

,id,date,store_nbr,family,onpromotion
0,3000888,2017-08-16,1,AUTOMOTIVE,0
1,3000889,2017-08-16,1,BABY CARE,0
2,3000890,2017-08-16,1,BEAUTY,2
3,3000891,2017-08-16,1,BEVERAGES,20
4,3000892,2017-08-16,1,BOOKS,0
...,...,...,...,...,...
28507,3029395,2017-08-31,9,POULTRY,1
28508,3029396,2017-08-31,9,PREPARED FOODS,0
28509,3029397,2017-08-31,9,PRODUCE,1
28510,3029398,2017-08-31,9,SCHOOL AND OFFICE SUPPLIES,9


In [ ]:
numeric_features = ["store_nbr", "onpromotion"]
categorical_features = ["family"]
drop_features = ["id", "date"]
transformer = make_column_transformer((StandardScaler(), numeric_features), (OneHotEncoder(sparse=False, handle_unknown='ignore'), categorical_features), ("drop", drop_features), n_jobs=-1)

In [ ]:
pipeline = make_pipeline(transformer)
pipeline.fit_transform(X_data, y_data)

array([[-1.70026736, -0.21342588,  1.        , ...,  0.        ,
         0.        ,  0.        ],
       [-1.70026736, -0.21342588,  0.        , ...,  0.        ,
         0.        ,  0.        ],
       [-1.70026736, -0.21342588,  0.        , ...,  0.        ,
         0.        ,  0.        ],
       ...,
       [-1.1869791 , 12.18646359,  0.        , ...,  1.        ,
         0.        ,  0.        ],
       [-1.1869791 ,  0.45683842,  0.        , ...,  0.        ,
         1.        ,  0.        ],
       [-1.1869791 , -0.21342588,  0.        , ...,  0.        ,
         0.        ,  1.        ]])

## Linear Regressor Method

In [ ]:
regressor = LinearRegression()
regressor.fit(pipeline.transform(X_data), y_data)

LinearRegression()

In [ ]:
results = regressor.predict(pipeline.transform(x_test))
id_column = []
for i in range(len(results)):
    id_column.append(3000888 + i)

results = pd.DataFrame(results, columns=['sales'])
results.insert(0, 'id', id_column, True)

# print(results.reset_index().to_string(index=False))
# results.index.name = None

# results.columns.name = {'id' : 'sales'}

# df.index = ['Row_1', 'Row_2', 'Row_3', 'Row_4']
# results.columns.names = {'id' : 'sales'}
# results.index.names = ['id']
#results.drop('')
#results.rename(columns={'':'id'}, inplace=True)#.columns = ["id", "sales"]#.rename(columns={0:'id', 1:'sales'}) # .replace(0, id_col)
results.to_csv('predictions2.csv', index=False)
results

,id,sales
0,3000888,-70.453857
1,3000889,-76.009644
2,3000890,-34.333374
3,3000891,2490.097168
4,3000892,-75.137451
...,...,...
28507,3029395,268.957275
28508,3029396,36.388306
28509,3029397,1062.317139
28510,3029398,123.655518
